# Incrementally refine a solution

This solution involved an iterative refinement of a solution to update BigQuery from an Airtable table. Airtable is a great and simple tool for capturing data and building data-driven solutions. BigQuery is a serverless data warehouse. I worked with these tools a lot in a previous role.

## ChatGPT Ask

Create a python program to update a bigquery table from aan Airtable table when values in Airtable are different from values in bigquery for a record.

In [ ]:
import airtable
from google.cloud import bigquery

# Set up Airtable credentials
AIRTABLE_API_KEY = 'your_airtable_api_key'
AIRTABLE_BASE_KEY = 'your_airtable_base_key'
AIRTABLE_TABLE_NAME = 'your_airtable_table_name'

# Set up BigQuery credentials
BQ_PROJECT_ID = 'your_bigquery_project_id'
BQ_DATASET_ID = 'your_bigquery_dataset_id'
BQ_TABLE_ID = 'your_bigquery_table_id'

# Set up Airtable client
airtable_client = airtable.Airtable(AIRTABLE_BASE_KEY, AIRTABLE_TABLE_NAME, AIRTABLE_API_KEY)

# Set up BigQuery client
bigquery_client = bigquery.Client()

# Retrieve data from Airtable
airtable_data = airtable_client.get_all()

# Retrieve data from BigQuery
bq_table = bigquery_client.get_table(f'{BQ_PROJECT_ID}.{BQ_DATASET_ID}.{BQ_TABLE_ID}')
bq_data = bigquery_client.list_rows(bq_table).to_dataframe()

# Find records that need to be updated in BigQuery
records_to_update = []
for airtable_record in airtable_data:
    airtable_record_id = airtable_record['id']
    for bq_index, bq_record in bq_data.iterrows():
        bq_record_id = bq_record['id']
        if airtable_record_id == bq_record_id:
            for airtable_field, airtable_value in airtable_record['fields'].items():
                bq_value = bq_record[airtable_field]
                if airtable_value != bq_value:
                    records_to_update.append({'id': bq_record_id, airtable_field: airtable_value})

# Update records in BigQuery
if len(records_to_update) > 0:
    rows_to_update = [bigquery_client.update_row(bq_table, record['id'], record) for record in records_to_update]
    print(f'Updated {len(rows_to_update)} rows in BigQuery.')
else:
    print('No rows need to be updated in BigQuery.')


## ChatGPT Ask

use functions for the last request

In [ ]:
import airtable
from google.cloud import bigquery

# Set up Airtable credentials
AIRTABLE_API_KEY = 'your_airtable_api_key'
AIRTABLE_BASE_KEY = 'your_airtable_base_key'
AIRTABLE_TABLE_NAME = 'your_airtable_table_name'

# Set up BigQuery credentials
BQ_PROJECT_ID = 'your_bigquery_project_id'
BQ_DATASET_ID = 'your_bigquery_dataset_id'
BQ_TABLE_ID = 'your_bigquery_table_id'

def get_airtable_data():
    airtable_client = airtable.Airtable(AIRTABLE_BASE_KEY, AIRTABLE_TABLE_NAME, AIRTABLE_API_KEY)
    return airtable_client.get_all()

def get_bigquery_data():
    bigquery_client = bigquery.Client()
    bq_table = bigquery_client.get_table(f'{BQ_PROJECT_ID}.{BQ_DATASET_ID}.{BQ_TABLE_ID}')
    return bigquery_client.list_rows(bq_table).to_dataframe()

def find_records_to_update(airtable_data, bq_data):
    records_to_update = []
    for airtable_record in airtable_data:
        airtable_record_id = airtable_record['id']
        for bq_index, bq_record in bq_data.iterrows():
            bq_record_id = bq_record['id']
            if airtable_record_id == bq_record_id:
                for airtable_field, airtable_value in airtable_record['fields'].items():
                    bq_value = bq_record[airtable_field]
                    if airtable_value != bq_value:
                        records_to_update.append({'id': bq_record_id, airtable_field: airtable_value})
    return records_to_update

def update_bigquery(records_to_update):
    if len(records_to_update) > 0:
        bigquery_client = bigquery.Client()
        bq_table = bigquery_client.get_table(f'{BQ_PROJECT_ID}.{BQ_DATASET_ID}.{BQ_TABLE_ID}')
        rows_to_update = [bigquery_client.update_row(bq_table, record['id'], record) for record in records_to_update]
        print(f'Updated {len(rows_to_update)} rows in BigQuery.')
    else:
        print('No rows need to be updated in BigQuery.')

def main():
    airtable_data = get_airtable_data()
    bq_data = get_bigquery_data()
    records_to_update = find_records_to_update(airtable_data, bq_data)
    update_bigquery(records_to_update)

if __name__ == '__main__':
    main()


## ChatGPT Ask

refactor find_records_to_update to not be so nested

### Note: Only the function I referred to was returned

In [ ]:
def find_records_to_update(airtable_data, bq_data):
    records_to_update = []
    for airtable_record in airtable_data:
        airtable_record_id = airtable_record['id']
        bq_record = bq_data[bq_data['id'] == airtable_record_id].iloc[0]
        for airtable_field, airtable_value in airtable_record['fields'].items():
            bq_value = bq_record[airtable_field]
            if airtable_value != bq_value:
                records_to_update.append({'id': airtable_record_id, airtable_field: airtable_value})
    return records_to_update


## ChatGPT Ask

show me all the code

In [ ]:
import airtable
from google.cloud import bigquery

# Set up Airtable credentials
AIRTABLE_API_KEY = 'your_airtable_api_key'
AIRTABLE_BASE_KEY = 'your_airtable_base_key'
AIRTABLE_TABLE_NAME = 'your_airtable_table_name'

# Set up BigQuery credentials
BQ_PROJECT_ID = 'your_bigquery_project_id'
BQ_DATASET_ID = 'your_bigquery_dataset_id'
BQ_TABLE_ID = 'your_bigquery_table_id'


def get_airtable_data():
    airtable_client = airtable.Airtable(AIRTABLE_BASE_KEY, AIRTABLE_TABLE_NAME, AIRTABLE_API_KEY)
    return airtable_client.get_all()


def get_bigquery_data():
    bigquery_client = bigquery.Client()
    bq_table = bigquery_client.get_table(f'{BQ_PROJECT_ID}.{BQ_DATASET_ID}.{BQ_TABLE_ID}')
    return bigquery_client.list_rows(bq_table).to_dataframe()


def find_records_to_update(airtable_data, bq_data):
    records_to_update = []
    for airtable_record in airtable_data:
        airtable_record_id = airtable_record['id']
        bq_record = bq_data[bq_data['id'] == airtable_record_id].iloc[0]
        for airtable_field, airtable_value in airtable_record['fields'].items():
            bq_value = bq_record[airtable_field]
            if airtable_value != bq_value:
                records_to_update.append({'id': airtable_record_id, airtable_field: airtable_value})
    return records_to_update


def update_bigquery(records_to_update):
    if len(records_to_update) > 0:
        bigquery_client = bigquery.Client()
        bq_table = bigquery_client.get_table(f'{BQ_PROJECT_ID}.{BQ_DATASET_ID}.{BQ_TABLE_ID}')
        rows_to_update = [bigquery_client.update_row(bq_table, record['id'], record) for record in records_to_update]
        print(f'Updated {len(rows_to_update)} rows in BigQuery.')
    else:
        print('No rows need to be updated in BigQuery.')


def main():
    airtable_data = get_airtable_data()
    bq_data = get_bigquery_data()
    records_to_update = find_records_to_update(airtable_data, bq_data)
    update_bigquery(records_to_update)


if __name__ == '__main__':
    main()


## Observations

In [ ]:
# We can incrementally refine a solution
# 